## Design matrices

want to have a simple formula-like system where you can say

```
Design(1+V(x)+V(y**2)+C(a)+C(a)&C(b)+C(d)*C(e)+V(x)%C(a))
```
and a design matrix will be built from it.

C is main effect (factor), V is variate, 1 is the intercept, * does main effects % just interactions


In [229]:
class Term:
    def __init__(self, value):
        self.value = value
        
    def __add__(self, other):
        return Sum(self, asterm(other))
    
    def __radd__(self, other):
        return Sum(asterm(other), self)
    
    def __mod__(self, other):
        M = Interaction(self, asterm(other))
        return M if len(M.value)>1 else M.value[0]
    
    def __mul__(self, other):
        other = asterm(other)
        return self+other+self%other
    
    def __pow__(self, n):
        s = self
        for i in range(1, n):
            s = s*self
        return s
    
    def __eq__(self, other):
        return self.__class__ == other.__class__ and self.value == other.value
    
    def __lt__(self, other):
        if self.__class__ != other.__class__: 
            if isinstance(self, V): return True # VC or VI
            if isinstance(self, Interaction): return False # IC or IV
            if isinstance(other, V): return False # CV
            return True # CI
        return self.value<other.value
    
def listif(x, cls):
    return x.value if isinstance(x, cls) else [x]

def asterm(x):
    return x if isinstance(x, Term) else Term(x)
    
class Sum(Term):
    def __init__(self, a, b):
        # sum flattens sums
        values = unique(listif(a, Sum)+listif(b, Sum))
        super().__init__(values)
        
    def __mod__(self, other):
        # distribute the interaction over sums
        if isinstance(other, Sum):
            values = [a%b for a in self.value for b in other.value]
        else:
            values = [a%other for a in self.value]
        s = values[0]+values[1]
        for v in values[2:]:
            s = s+v
        return s
        
    def __eq__(self, other):
        return self.__class__ == other.__class__ and sorted(self.value) == sorted(other.value)
    
    
class V(Term):
    pass

class C(Term):
    pass
        
def unique(x):
    y = []
    for a in x:
        if a not in y:
            y.append(a)
    return y

class Interaction(Term):
    def __init__(self, a, b):
        values = unique(listif(a, Interaction)+listif(b, Interaction))
        super().__init__(values)

    def __eq__(self, other):
        return self.__class__ == other.__class__ and sorted(self.value) == sorted(other.value)
    

In [230]:
import numpy as np
a,b,c,d = 'abcd'

(1+C(c)*V(d)%V(a)).value

In [226]:
def printexpr(E):
    if isinstance(E, Sum):
        return '+'.join(map(printexpr, E.value))
    if isinstance(E, Interaction):
        return '%'.join(map(printexpr, E.value))
    return E.__class__.__name__+'('+str(E.value)+')'

In [237]:
printexpr(-1+(1+C(c)+V(a))*V(b))

'Term(-1)+Term(1)+C(c)+V(a)+V(b)+Term(1)%V(b)+C(c)%V(b)+V(a)%V(b)'

In [228]:
printexpr((C(a)+C(b) +V(d))**3)

'C(a)+C(b)+V(d)+C(a)%C(b)+C(a)%V(d)+C(b)%V(d)+C(a)%C(b)%V(d)'